In [1]:
import configparser
from pathlib import Path
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
from ci_adapt_utilities import *
import pickle
import os
from matplotlib import pyplot as plt
import ast
# MIRACA color scheme

# Load default configuration and parameters
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'
color_string = config.get('DEFAULT', 'miraca_colors')
miraca_colors = ast.literal_eval(color_string)

In [2]:
# if collected_output variable doesnt exist, load from pickle file
if 'collect_output' not in locals():
    collect_output_path = f'{interim_data_path}/sample_collected_run.pkl'
    with open(collect_output_path, 'rb') as f:
        collect_output = pickle.load(f)

In [3]:
# Calculate the expected annual damages (EAD)
summed_output = {}
# Iterate over the items in the collect_output dictionary
for hazard_map, asset_dict in collect_output.items():
    # If the hazard_map and hazard_curve combination is not already in the summed_output dictionary, add it with the sum of the current lower and upper bounds
    if hazard_map not in summed_output:
        summed_output[hazard_map] = (sum(value[0] for value in asset_dict.values()), sum(value[1] for value in asset_dict.values()))
    # If the hazard_map and hazard_curve combination is already in the summed_output dictionary, add the sum of the current lower and upper bounds to the existing ones
    else:
        summed_output[hazard_map][0] += sum(value[0] for value in asset_dict.values())
        summed_output[hazard_map][1] += sum(value[1] for value in asset_dict.values())


In [4]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets = preprocess_assets(assets_path)
asset_options={'bridge_design_rp':'M',
               'tunnel_design_rp':'M'}

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()
type_dict = assets['asset'].to_dict()

print(f"{len(assets)} assets loaded.")

122720 assets loaded.


In [15]:
area_of_interest = data_path / 'visualisation' / 'rhineland_palatinate.geojson'
area_of_interest = gpd.read_file(area_of_interest)
area_of_interest = area_of_interest.to_crs(epsg=4326)

In [5]:
# Extract the geometries of the stretches of disrupted rail track
disrupted_asset_ids=[]
for hazard_map, asset_dict in collect_output.items():
    rp = hazard_map.split('_RW_')[-1].split('_')[0]
    if rp != 'L': continue

    overlay_assets=load_baseline_run(hazard_map, interim_data_path, only_overlay=True)
    disrupted_asset_ids.extend(overlay_assets.asset.unique())



In [6]:
#plot the disrupted rail track
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gpd

haz_map_path=Path(r'C:\Data\Floods\Germany\raw_data\SzenarioSelten\DERP_RW_L.shp')
haz_map=gpd.read_file(haz_map_path)
haz_map=gpd.GeoDataFrame(haz_map, geometry='geometry')

In [7]:

haz_map['flood_area']=int(str(haz_map['T_class'][0])[0])
haz_map['depth_class']=int(str(haz_map['T_class'][0])[-1])   
haz_map=haz_map.to_crs(4326)

In [21]:
flood_gdf = haz_map
# helper function
def zoomlevel_from_deg(deg): #https://stackoverflow.com/questions/30052990/how-to-use-openstreetmap-background-on-matplotlib-basemap
    "Calculate OSM zoom level from a span in degrees.  Adjust +/-1 as desired"
    from numpy import log2, clip, floor
    zoomlevel = int(clip(floor(log2(360) - log2(delta)),0,20 ))
    return zoomlevel 

extent = haz_map.total_bounds

# Add clearance around the basin
clearance = 0.1  
lon_min, lat_min, lon_max, lat_max = extent
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min

lon_min -= clearance * lon_range
lat_min -= clearance * lat_range
lon_max += clearance * lon_range
lat_max += clearance * lat_range

# Calculate aspect ratio
aspect_ratio = lon_range / lat_range

lon_i = lon_min + (lon_max - lon_min) / 2
lat_i = lat_min + (lat_max - lat_min) / 2
delta = 0.0016 # 0.0012 # 38 to 0.0002 degrees to avoid north pole
zoom = zoomlevel_from_deg(delta)-8 # 10 #  0-19 
print(f"Zoom Level: {zoom}")

# Create a plot with the OSM tiles
print('Requesting OSM tiles...')
request_osm = cimgt.OSM()

fig = plt.figure(figsize=(30 * aspect_ratio, 30))
ax = plt.axes(projection=request_osm.crs)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_image(request_osm, zoom, alpha=0.5)  # Adjust zoom level as needed

# Plot the assets
print('Plotting assets...')
plot_assets=assets.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets.plot(ax=ax, color='grey', markersize=10, transform=ccrs.PlateCarree())

# Plot the affected assets
plot_assets_affected = assets.iloc[disrupted_asset_ids].drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_affected.plot(ax=ax, color='red', markersize=10, transform=ccrs.PlateCarree())


# Plot the flood areas
print('Plotting flood areas...')
f_area_colors = {1:'blue', 3:'green'}
for f_area in tqdm(flood_gdf.flood_area.unique()):
    print('f_area: ',f_area)
    for f_depth in flood_gdf.depth_class.unique():
        print('->f_depth: ',f_depth)
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            subset_gdf.plot(ax=ax, facecolor=miraca_colors[color_key], edgecolor=miraca_colors[color_key], linewidth=2, alpha=0.9, transform=ccrs.PlateCarree())
            
    

print('Finalizing plot...')

ax.gridlines(draw_labels=True, xlabel_style={'size': 20}, ylabel_style={'size': 20})
plt.show()

Zoom Level: 9
Requesting OSM tiles...
Plotting assets...
Plotting flood areas...


  0%|          | 0/1 [00:00<?, ?it/s]

f_area:  1
->f_depth:  1


In [18]:
# save the plot as svg
plot_path = data_path / 'output' / 'plots' / 'disrupted_rail_track3.png'
fig.savefig(plot_path)

In [19]:
subset_gdf.head(3)

,Id,T_class,geometry,flood_area,depth_class
0,1,11,"MULTIPOLYGON (((8.23033 48.96809, 8.23032 48.9...",1,1
1,2,11,"MULTIPOLYGON (((8.29708 49.02565, 8.29708 49.0...",1,1
2,3,11,"MULTIPOLYGON (((8.23714 49.04183, 8.23714 49.0...",1,1


In [12]:
assets.head(3)

,osm_id,asset,name,gauge,electrified,voltage,bridge,maxspeed,service,tunnel,other_tags,railway:traffic_mode,usage,geometry,buffered
0,3568324,rail,None,1435,contact_line,15000,yes,None,None,None,"""detail""=>""track"",""frequency""=>""16.7"",""layer""=...",None,"""main""","LINESTRING (960271.182 6414299.927, 960265.305...","POLYGON ((960265.3054722614 6414279.897151432,..."
1,3568326,rail,None,1435,contact_line,15000,yes,160,None,None,"""frequency""=>""16.7"",""layer""=>""1"",""level""=>""1"",...",None,"""main""","LINESTRING (958329.960 6399001.946, 958334.401...","POLYGON ((958334.4021388514 6398969.107017589,..."
2,3568327,rail,None,1435,contact_line,15000,None,160,None,None,"""frequency""=>""16.7"",""operator""=>""DB Netz"",""pas...",None,"""main""","LINESTRING (958811.238 6394828.612, 958820.534...","POLYGON ((958820.5343572184 6394747.648176393,..."
